# Blog Post 3: Color Transfer, UNet, and Semantic Segmentation

In this post, we'll go over the following key points:

1. comment on how we handle the various modalities
1. overview of our Neural Network used for segmentation
1. discuss our plan moving forward with Semantic Segmentation

### reading data and preprocessing

In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["text.usetex"] = True
plt.rcParams["figure.dpi"] = 125

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from deton8.utils import DataReader

In [3]:
training = DataReader("../data/training/")
train_metadata, x_train = training.get()

y_train = np.stack(training.masks)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train_metadata, val_metadata, x_train, x_val, y_train, y_val = train_test_split(train_metadata, x_train, y_train, train_size = 0.8)

In [7]:
x_dev = x_train[::10]